In [1]:
original_prica = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [2]:
from collections import defaultdict

In [3]:
price = defaultdict(int)

In [4]:
for i,p in enumerate(original_prica):
    price[i+1] = p

In [5]:
price[11]

35

## Get the max splitting by enumerate

In [6]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

In [7]:
def r(n):
    
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)]
    )

In [8]:
r(10)

30

In [9]:
r(15)

45

In [10]:
import time

In [11]:
#@get_time
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [12]:
start = time.time()
print(fibonacci(34))
end = time.time()
print(end-start)

5702887
1.806950330734253


In [13]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [14]:
start = time.time()
print(fibonacci_op(64))
end = time.time()
print(end-start)

14662949395604
0.0029981136322021484


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [15]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [16]:
def f1(func):
    def wrapper(*args,**kwargs):
        print('Started')
        func(*args,**kwargs)
        print('Ended')
    return wrapper

In [17]:
def f():
    print('HELLO')

In [18]:
f = f1(f)

In [19]:
print(f.__name__)

wrapper


In [20]:
@f1
def g(a):
    print(a)

In [21]:
g('hello')

Started
hello
Ended


In [22]:
def k(*arg,**kwargs):
    print(kwargs)

In [23]:
k(6,b=5)

{'b': 5}


In [24]:
from functools import wraps

In [25]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

# We use this method to solve Cut Rod probelm¶

In [26]:
solution = {}

In [27]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [28]:
r(20)

60

In [29]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35,
             15: 0,
             14: 0,
             13: 0,
             12: 0,
             20: 0,
             19: 0,
             18: 0,
             17: 0,
             16: 0})

In [30]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

# How do we parse solution?¶

In [31]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [32]:
r(24)

75

In [33]:
parse_solution(20)

[11, 6, 3]

In [34]:
r(50)

157

In [35]:
parse_solution(50)

[11, 11, 11, 11, 6]

# Edit Distance

In [36]:
solution = {}

In [37]:
from functools import lru_cache

In [38]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    #print('({},{}),{}'.format(string1, string2, candidates))
    return min_distance

In [39]:
edit_distance('ABCDECG','ABCCEF')

3

In [40]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDEC', 'A'): 'DEL C',
 ('ABCDEC', 'AB'): 'DEL C',
 ('ABCDEC', 'ABC'): 'DEL C',
 ('ABCDEC', 'ABCC'): '',
 ('ABCDEC', 'ABCCE'): 'DEL C',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'A'): 'DEL G',
 ('ABCDECG', 'A

## Todo: Parse Solution is our homework

In [41]:
import string
parse = []
def parse_solution(string1, string2):
    if string1 == string2:
        return parse
    if 'ADD' in solution[(string1, string2)]:
        parse.append(solution[(string1, string2)])
        parse_solution(string1, string2[:-1])
    elif 'DEL' in solution[(string1, string2)]:
        parse.append(solution[(string1, string2)])
        parse_solution(string1[:-1], string2)
    else:
        parse.append(solution[(string1, string2)])
        parse_solution(string1[:-1], string2[:-1])

In [42]:
parse_solution('ABCDECG','ABCCEF')

In [43]:
parse

['DEL G', 'SUB C => F', '', 'SUB D => C']

# Problem Case 3: Pinyin Auto Correction Problem

In [3]:
chinese_dataset = 'article_9k.txt'

In [4]:
CHINESE_CHARATERS = open(chinese_dataset).read()

In [5]:
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [6]:
!pip install pinyin

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [7]:
import pinyin

In [8]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [9]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [10]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [11]:
len(CHINESE_CHARATERS_COPYS)

129433034

In [12]:
import re

In [13]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [14]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [15]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [16]:
from collections import Counter, defaultdict

In [17]:
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [18]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [19]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [20]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [21]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [22]:
print(edits0('pinyin'))

{'pinyin'}


In [23]:
print(edits1('pinyin'))

{'pinysn', 'pinnyin', 'tpinyin', 'pniyin', 'pinyimn', 'pinzyin', 'pminyin', 'pinwin', 'cinyin', 'pindin', 'pinein', 'pianyin', 'pjinyin', 'pinyinj', 'painyin', 'pwinyin', 'pinrin', 'pinyion', 'fpinyin', 'ptnyin', 'piqyin', 'psnyin', 'piunyin', 'pinyibn', 'pinyiyn', 'pinhin', 'pinsin', 'pbinyin', 'dinyin', 'pinyib', 'qpinyin', 'pinyoin', 'pvinyin', 'pinxin', 'pinyif', 'pifnyin', 'pinygn', 'pingyin', 'piniyin', 'piyin', 'pinytin', 'opinyin', 'binyin', 'pinynin', 'pinnin', 'pinyln', 'pinyiwn', 'pinkyin', 'pincin', 'pinyyn', 'penyin', 'piynyin', 'pinycin', 'poinyin', 'inyin', 'punyin', 'pginyin', 'qinyin', 'pivnyin', 'pinyinc', 'linyin', 'pinyink', 'pinyins', 'pxinyin', 'pinyint', 'pinfin', 'pinyjin', 'pixnyin', 'pinyiz', 'piniyn', 'pinyiny', 'pinyinv', 'pinhyin', 'pixyin', 'pinyijn', 'pibnyin', 'pjnyin', 'pinyifn', 'pinlin', 'pihyin', 'pnnyin', 'pinyit', 'pinyni', 'pinyir', 'pgnyin', 'jinyin', 'pinyix', 'pinyip', 'pzinyin', 'kpinyin', 'pinyjn', 'lpinyin', 'pinvyin', 'pinykn', 'pinybin', '

# Test

In [24]:
correct('yin')

'yin'

In [25]:
correct('yign')

'ying'

In [26]:
correct('yinn')

'ying'

In [27]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [28]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [29]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP

In [30]:
PINYIN_COUNT.keys()

dict_keys(['ci', 'wai', 'zi', 'ben', 'zhou', 'yue', 'ri', 'qi', 'chu', 'xiao', 'mi', 'shou', 'ji', 'deng', 'kuan', 'xing', 'yu', 'yi', 'zan', 'ting', 'geng', 'xin', 'fa', 'bu', 'han', 'kai', 'ban', 'ti', 'yan', 'nei', 'ce', 'wen', 'ding', 'ying', 'xiang', 'que', 'bao', 'gong', 'cheng', 'shi', 'ke', 'zhong', 'quan', 'jing', 'li', 'jin', 'xi', 'tong', 'you', 'hua', 'zuo', 'ren', 'cai', 'zhe', 'ye', 'jiang', 'zhu', 'yao', 'yong', 'dao', 'm', 'i', 'u', 'de', 'zhi', 'qu', 'nian', 'ju', 'hou', 'huan', 'dai', 'le', 'dang', 'ran', 'guan', 'qie', 'wo', 'men', 'fang', 'long', 'wei', 'guo', 'w', 'n', 'd', 'o', 's', 'zhuo', 'mian', 'ping', 'tai', 'zheng', 'a', 'r', 'gao', 'qiang', 'diao', 'hui', 'yin', 'kao', 'lv', 'neng', 'er', 'bi', 'he', 'fan', 'ta', 'lian', 'ruan', 'zhao', 'jian', 'gu', 'hao', 'wan', 'mei', 'an', 'na', 'xie', 'yuan', 'ma', 'bian', 'di', 'jie', 'b', 'g', 'l', 't', 'e', 'jia', 'gou', 'liao', 'xian', 'c', 'p', 'lan', 'ya', 'f', 'chuan', 'sheng', 'shao', 'kong', 'q', 'shuo', 'pu',

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

In [31]:
length = len(tokens(CHINESE_CHARATERS_COPYS))

In [32]:
def pro(w1, w2):
    if w1 + w2 in PINYIN_COUNT.keys(): return PINYIN_COUNT[w1+w2]/(PINYIN_COUNT[w1+w2] + PINYIN_COUNT[w1])
    else: return 1/length

In [33]:
pro('q', 'i')

0.9966113531573142

In [34]:
pro('w','oyaoshangqinghua')

3.197567086316437e-08

In [35]:
pro('wo','yaoshangqinghua')

3.197567086316437e-08

In [36]:
pro('y','aoshangqinghua')

3.197567086316437e-08

In [37]:
pro('y','a')

0.9698762775685853

In [38]:
pro('o','s')

3.197567086316437e-08

In [39]:
pro('y', 'a')

0.9698762775685853

In [40]:
pro('y','ao')

0.97923762060373

In [41]:
def split(string: str):
    length = len(string)
    matrix = [[0 for _ in range(length)] for _ in range(length)]
    index = -1
    new_string = ''
    for row in range(length):
        row = index + 1
        if row < length:
            for col in range(length):
                if row < col:
                    matrix[row][col] = pro(string[row], string[row+1:col+1])
            index = matrix[row].index(max(matrix[row]))
            new_string += string[row:index+1] + ' '
    return new_string

In [42]:
split('woyaoshangqinghua')

'wo yao shang qi ng hua '

贪心算法失败。接下来使用DP算法

In [72]:
def gen_matrix(string: str):
    length = len(string)
    matrix = [[0 for _ in range(length)] for _ in range(length)]
    for row in range(length):
        for col in range(length):
            if row < col:
                 matrix[row][col] = pro(string[row], string[row+1:col+1])
    return matrix

In [73]:
matrix = gen_matrix('woyaoshangqinghua')

In [74]:
matrix

[[0,
  0.9501564956724906,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08],
 [0,
  0,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08],
 [0,
  0,
  0,
  0.9698762775685853,
  0.97923762060373,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.197567086316437e-08,
  3.

In [878]:
matrix[0][1]*matrix[2][4]*matrix[5][9]*matrix[10][13]*matrix[14][16]

0.8442518048831049

In [883]:
matrix[0][1]+matrix[2][4]+matrix[5][9]+matrix[10][13]+matrix[14][16]

4.834561992617978

In [77]:
solution = {}
def get(string:str):
    matrix = gen_matrix(string)
    value, left, right = max([(matrix[0][-1], string, 0)] + [(get(string[0:n]) * get(string[n:]), string[0:n], string[n:]) for n in range(1,len(matrix)) if string[0:n] and string[n:]], key = lambda x: x[0])
    solution[string] = (left, right)
    return value

In [78]:
get('woyaoshangqinghua')

0.844251804883105

In [79]:
solution

{'w': ('w', 0),
 'o': ('o', 0),
 'y': ('y', 0),
 'a': ('a', 0),
 's': ('s', 0),
 'h': ('h', 0),
 'n': ('n', 0),
 'g': ('g', 0),
 'q': ('q', 0),
 'i': ('i', 0),
 'u': ('u', 0),
 'ua': ('ua', 0),
 'hu': ('hu', 0),
 'hua': ('hua', 0),
 'gh': ('gh', 0),
 'ghu': ('ghu', 0),
 'ghua': ('ghua', 0),
 'ng': ('ng', 0),
 'ngh': ('ngh', 0),
 'nghu': ('nghu', 0),
 'nghua': ('nghua', 0),
 'in': ('in', 0),
 'ing': ('ing', 0),
 'ingh': ('ingh', 0),
 'inghu': ('inghu', 0),
 'inghua': ('inghua', 0),
 'qi': ('qi', 0),
 'qin': ('qin', 0),
 'qing': ('qing', 0),
 'qingh': ('qingh', 0),
 'qinghu': ('qing', 'hu'),
 'qinghua': ('qing', 'hua'),
 'gq': ('gq', 0),
 'gqi': ('gqi', 0),
 'gqin': ('gqin', 0),
 'gqing': ('gqing', 0),
 'gqingh': ('gqingh', 0),
 'gqinghu': ('gqinghu', 0),
 'gqinghua': ('gqinghua', 0),
 'ngq': ('ngq', 0),
 'ngqi': ('ngqi', 0),
 'ngqin': ('ngqin', 0),
 'ngqing': ('ngqing', 0),
 'ngqingh': ('ngqingh', 0),
 'ngqinghu': ('ngqinghu', 0),
 'ngqinghua': ('ngqinghua', 0),
 'an': ('an', 0),
 'ang'

In [80]:
def parse_solution(string:str, solution = solution):
    left, right = solution[string]
    if right == 0: return left+' '
    
    return parse_solution(left) + parse_solution(right)
        

In [81]:
parse_solution('woyaoshangqinghua')

'wo yao shang qing hua '

In [100]:
def auto_split_pinyin(string:str):
    value = get(string)
    result = parse_solution(string, solution = solution)
    return result

In [101]:
auto_split_pinyin('woyaoshangqinghua')

'wo yao shang qing hua '

In [102]:
auto_split_pinyin('laizheheren')

'lai zhe he ren '

In [103]:
auto_split_pinyin('baoshangminglai')

'bao shang ming lai '

In [104]:
auto_split_pinyin('shangshanruoshui')

'shang shan ruo shui '

In [105]:
auto_split_pinyin('haohaoxuexi')

'hao hao xue xi '

In [106]:
auto_split_pinyin('tiantianxiangshang')

'tian tian xiang shang '